# Study on Yelp-Hat

Paper: https://davis.wpi.edu/dsrg/PROJECTS/YELPHAT/2020_ACL_Human_vs_Machine-2.pdf

Summary:
* __Do annotators carefully choose relevant words?__ Yes, as the collecting time and number of chosen words increase accross the sentence length.

Sigles:
* __HAM__ (Human Attention Map): what annotators denote
* __CAM__ (Consensus Attention Map): bitwise __AND__ operation of the HAMs
* __SAM__ (Super Attention Map): bitwise __OR__ operation of the HAMs. 


In [1]:
%load_ext autoreload
%autoreload 2

from IPython.display import display, HTML
import os
from os import path

import sys
sys.path.append("./../src")

cache_path = path.join(os.getcwd(), '..', '.cache')

DATASET_NAME='yelp-hat'

dataset_path = path.join(cache_path, 'dataset', DATASET_NAME)

tmp_path = path.join('.cache', '2022-07-29')
os.makedirs(tmp_path,exist_ok=True)

Download and extract dataset

In [2]:
from torchtext.utils import download_from_url, extract_archive
import shutil

# Download dataset zip
URL='https://github.com/cansusen/Human-Attention-for-Text-Classification/archive/205c1552bc7be7ec48623d79d85d4c6fbfe62362.zip'

zip_path = download_from_url(URL, root=dataset_path, path=path.join(dataset_path, f'{DATASET_NAME}.zip'))
extracted_path = path.join(dataset_path, 'caching')
files = extract_archive(from_path=zip_path, to_path=extracted_path)
files = [f for f in files if f.endswith('.csv')]

for f in files: shutil.copy2(f, extracted_path)

A quoi ressemble la dataset?

In [3]:
import pandas as pd

df = pd.read_csv(path.join(extracted_path, 'ham_part1(50words).csv'))
df

,Input.label,Input.text,Answer.Q1Answer,Answer.html_output
0,1,Out in Twinsburg for work and wasn't expecting...,yes,<span>Out</span> <span>in</span> <span>Twinsbu...
1,1,Out in Twinsburg for work and wasn't expecting...,yes,<span>Out</span> <span>in</span> <span>Twinsbu...
2,1,Out in Twinsburg for work and wasn't expecting...,yes,<span>Out</span> <span>in</span> <span>Twinsbu...
3,0,Very slow. Never been in the drive at any othe...,no,"<span class=""active"">Very</span> <span class=""..."
4,0,Very slow. Never been in the drive at any othe...,no,"<span>Very</span> <span class=""active"">slow.</..."
...,...,...,...,...
895,0,I went here to get a snack before I went on th...,no,<span>I</span> <span>went</span> <span>here</s...
896,0,I went here to get a snack before I went on th...,no,<span>I</span> <span>went</span> <span>here</s...
897,0,Always packed for lunch. Probably because Pit...,no,<span>Always</span> <span>packed</span> <span>...
898,0,Always packed for lunch. Probably because Pit...,no,<span>Always</span> <span>packed</span> <span>...


In [5]:
import re
def generate_binary_human_attention_vector(html, num_words_in_review, max_words):
    # Function provided by the dataset :
    # https://github.com/cansusen/Human-Attention-for-Text-Classification/blob/master/generate_ham/sample_generate.ipynb

    p = re.compile('<span(.*?)/span>')
    all_span_items = p.findall(html)

    if html == '{}':
        print('Empty human annotation - This should never print')
        return [0] * max_words

    if len(all_span_items) == num_words_in_review + 1:
        if (all_span_items[num_words_in_review] == '><') or (all_span_items[num_words_in_review] == ' data-vivaldi-spatnav-clickable="1"><'):

            binarized_human_attention = [0] * max_words
            for i in range(0, len(all_span_items) - 1):
                if 'class="active"' in all_span_items[i]:
                    binarized_human_attention[i] = 1

        else:
            print('This should never print.')
    else:
        print('This should never print.')

    return binarized_human_attention

MAX_WORDS = 100
i = 0
html = df['Answer.html_output'][i]
num_highlighted = html.count('class="active"')
num_words_in_review = len(df['Input.text'][i].split())

binarized_human_attention = generate_binary_human_attention_vector(html, num_words_in_review, MAX_WORDS)


print("Number of words highlighted in this review:",num_highlighted)
print("Original annotation:", html)
print("Binarized attention map:",binarized_human_attention)

Number of words highlighted in this review: 6
Original annotation: <span>Out</span> <span>in</span> <span>Twinsburg</span> <span>for</span> <span>work</span> <span>and</span> <span>wasn't</span> <span>expecting</span> <span>to</span> <span>find</span> <span>a</span> <span>well</span> <span>reviewed</span> <span>sushi</span> <span>restaurant</span> <span>but</span> <span class="active">glad</span> <span>I</span> <span>did.</span> <span>It</span> <span>was</span> <span>quite</span> <span>busy</span> <span>for</span> <span>a</span> <span>Monday</span> <span>and</span> <span>the</span> <span>poor</span> <span>waitress</span> <span>was</span> <span>slammed</span> <span>but</span> <span>the</span> <span>sushi</span> <span>chef</span> <span>stepped</span> <span>in</span> <span>to</span> <span>help</span> <span>and</span> <span>was</span> <span class="active">very</span> <span class="active">friendly.</span> <span>The</span> <span class="active">presentation</span> <span>and</span> <span class

In [137]:
def tokenize(html):
    p = re.compile(r'<span[^>]*>(.+?)</span>')
    return p.findall(html)

def human_attention(html):

    p = re.compile('<span(.*?)/span>')
    all_span_items = p.findall(html)
    if all_span_items[-1] == '><': all_span_items = all_span_items[:-1]

    return ['class="active"' in span_item for span_item in all_span_items]

v_hat = human_attention(html)
print(v_hat)

# Check if we tokenize html, can our function "human_attention" reproduce the exact same length
for fpath in os.listdir(dataset_path):
    
    if fpath.endswith('.csv'):
        print('Check',fpath)
        df = pd.read_csv(path.join(dataset_path, fpath))
        print(len(df['Answer.html_output']))
        for html in df['Answer.html_output']:
            tokens = tokenize(html)
            v_hat = human_attention(html)
            if len(tokens) != len(v_hat):
                print(len(tokens), len(v_hat))
                print(' '.tokens)
                display(HTML(html))

NameError: name 'html' is not defined

Proceed dataset step by step

In [148]:
from data.yelp_hat.utils import yelp_hat_ham, yelp_hat_token
import numpy as np
import spacy

nlp = spacy.load('en_core_web_sm')

df = pd.read_csv(path.join(extracted_path, 'ham_part5.csv'))
display(HTML('<h3>Raw dataset</h3>'))
display(df.head())

dupli_2 = duplicates[duplicates['size'] == 2]  # finds dupli_2
df = df[~df['Input.text'].isin(dupli_2['index'])]

df['ham_'] = df[f'Answer.html_output'].apply(lambda x: yelp_hat_ham(x, nlp)).apply(lambda x: np.array(x))
df['text_tokens'] = df['Answer.html_output'].apply(lambda x: yelp_hat_token(x, nlp))
df = df.rename(columns={'Answer.Q1Answer': 'human_label_', 'Input.text': 'text', 'Input.label': 'label'})

display(HTML('<h3>Preprocess text and human attention</h3>'))
display(df.head())

dfs = [df.loc[0::3, ['text', 'label', 'text_tokens']].reset_index(drop=True)]
for idx in range(3):
    _data = df.loc[idx::3, ['ham_', 'human_label_']]
    _data = _data.reset_index(drop=True).add_suffix(str(idx))
    dfs += [_data]
    
df = pd.concat(dfs, axis=1)

def cam(row):
    return np.logical_and(row['ham_0'],row['ham_1'],row['ham_2'])

def sam(row):
    return np.logical_or(row['ham_0'],row['ham_1'],row['ham_2'])

def ham(row):
    return ((row['ham_0'] + row['ham_1'] + row['ham_2'])/3 >= 0.5).astype(int)

df['cam'] = df.apply(cam,axis=1)
df['sam'] = df.apply(sam,axis=1)
df['ham'] = df.apply(ham,axis=1)

df.head()

,Input.label,Input.text,Answer.Q1Answer,Answer.html_output
0,0,"I don't think so. \n\nThis ""buffet"" is probabl...",no,<span>I</span> <span>don't</span> <span>think<...
1,0,"I don't think so. \n\nThis ""buffet"" is probabl...",no,"<span>I</span> <span class=""active"">don't</spa..."
2,0,"I don't think so. \n\nThis ""buffet"" is probabl...",no,<span>I</span> <span>don't</span> <span>think<...
3,1,Ate here two times now and food has been excel...,yes,<span>Ate</span> <span>here</span> <span>two</...
4,1,Ate here two times now and food has been excel...,yes,<span>Ate</span> <span>here</span> <span>two</...


,label,text,human_label_,Answer.html_output,ham_,text_tokens
0,0,"I don't think so. \n\nThis ""buffet"" is probabl...",no,<span>I</span> <span>don't</span> <span>think<...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[I, do, n't, think, so, ., This, "", buffet, "",..."
1,0,"I don't think so. \n\nThis ""buffet"" is probabl...",no,"<span>I</span> <span class=""active"">don't</spa...","[0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[I, do, n't, think, so, ., This, "", buffet, "",..."
2,0,"I don't think so. \n\nThis ""buffet"" is probabl...",no,<span>I</span> <span>don't</span> <span>think<...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[I, do, n't, think, so, ., This, "", buffet, "",..."
3,1,Ate here two times now and food has been excel...,yes,<span>Ate</span> <span>here</span> <span>two</...,"[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...","[Ate, here, two, times, now, and, food, has, b..."
4,1,Ate here two times now and food has been excel...,yes,<span>Ate</span> <span>here</span> <span>two</...,"[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...","[Ate, here, two, times, now, and, food, has, b..."


,text,label,text_tokens,ham_0,human_label_0,ham_1,human_label_1,ham_2,human_label_2,cam,sam,ham
0,"I don't think so. \n\nThis ""buffet"" is probabl...",0,"[I, do, n't, think, so, ., This, "", buffet, "",...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",no,"[0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",no,"[0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",no,"[0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,Ate here two times now and food has been excel...,1,"[Ate, here, two, times, now, and, food, has, b...","[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...",yes,"[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...",yes,"[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...",yes,"[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, ..."
2,Great food and great atmosphere. The one thing...,1,"[Great, food, and, great, atmosphere, ., The, ...","[1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",yes,"[1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, ...",yes,"[1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, ...",yes,"[1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, ...","[1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, ...","[1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, ..."
3,Obsessed. Atmosphere is a great combo of class...,1,"[Obsessed, ., Atmosphere, is, a, great, combo,...","[1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, ...",yes,"[1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, ...",yes,"[1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, ...",yes,"[1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, ...","[1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, ...","[1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, ..."
4,I spend a lot of time trying as many burger jo...,1,"[I, spend, a, lot, of, time, trying, as, many,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",yes,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",yes,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",yes,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [88]:
df = pd.read_csv(path.join(extracted_path, 'ham_part5.csv'))
duplicates = df.groupby(df['Input.text'].tolist(),as_index=False).size()
display(duplicates['size'].unique())
for dup_rows in [2, 3]:
    print('Duplicate of', dup_rows, ':', sum(duplicate['size'] == dup_rows))
    
dupli_2 = duplicates[duplicates['size'] == 2]  # finds dupli_2
df = df[~df['Input.text'].isin(dupli_2['index'])]
duplicate = df.groupby(df['Input.text'].tolist(),as_index=False).size()
display(duplicate['size'].unique())

In [143]:
df['coherent_ham_01'] = df['ham_0'].str.len() == df['ham_1'].str.len()
df['coherent_ham_12'] = df['ham_1'].str.len() == df['ham_2'].str.len()
print(df['coherent_ham_01'].all())
print(df['coherent_ham_12'].all())
print('#Incoherent length:', df['coherent_ham_12'].sum())

True
True
#Incoherent length: 999


In [81]:
def cam(row):
    return np.logical_and(row['ham_0'],row['ham_1'],row['ham_2'])

def sam(row):
    return np.logical_or(row['ham_0'],row['ham_1'],row['ham_2'])

def ham(row):
    return ((row['ham_0'] + row['ham_1'] + row['ham_2'])/3 >= 0.5).astype(int)

for f in files:
    print(f)
    df = pd.read_csv(f)
    df['ham_'] = df[f'Answer.html_output'].apply(yelp_hat_ham).apply(lambda x: np.array(x))
    df['text_tokens'] = df['Answer.html_output'].apply(yelp_hat_token)
    df = df.rename(columns={'Answer.Q1Answer': 'human_label_', 'Input.text': 'text', 'Input.label': 'label'})
    dfs = [df.loc[0::3, ['text', 'label', 'text_tokens']].reset_index(drop=True)]
    for idx in range(3):
        _data = df.loc[idx::3, ['ham_', 'human_label_']]
        _data = _data.reset_index(drop=True).add_suffix(str(idx))
        dfs += [_data]

    df = pd.concat(dfs, axis=1)

    try:
        df['cam'] = df.apply(cam,axis=1)
        df['sam'] = df.apply(sam,axis=1)
        df['ham'] = df.apply(ham,axis=1)
    except ValueError as e:
        print(e)
        continue

/Users/dunguyen/Projects/explanation_on_pair_sequences_task/notebooks/../.cache/dataset/yelp-hat/caching/Human-Attention-for-Text-Classification-205c1552bc7be7ec48623d79d85d4c6fbfe62362/raw_data/ham_part1(50words).csv
/Users/dunguyen/Projects/explanation_on_pair_sequences_task/notebooks/../.cache/dataset/yelp-hat/caching/Human-Attention-for-Text-Classification-205c1552bc7be7ec48623d79d85d4c6fbfe62362/raw_data/ham_part3.csv
/Users/dunguyen/Projects/explanation_on_pair_sequences_task/notebooks/../.cache/dataset/yelp-hat/caching/Human-Attention-for-Text-Classification-205c1552bc7be7ec48623d79d85d4c6fbfe62362/raw_data/ham_part4.csv
/Users/dunguyen/Projects/explanation_on_pair_sequences_task/notebooks/../.cache/dataset/yelp-hat/caching/Human-Attention-for-Text-Classification-205c1552bc7be7ec48623d79d85d4c6fbfe62362/raw_data/ham_part5.csv
operands could not be broadcast together with shapes (55,) (55,) (71,) 
/Users/dunguyen/Projects/explanation_on_pair_sequences_task/notebooks/../.cache/dat

In [116]:
import numpy as np

def cam(row):
    return np.logical_and(row['ham_0'],row['ham_1'],row['ham_2'])

def sam(row):
    return np.logical_or(row['ham_0'],row['ham_1'],row['ham_2'])

def ham(row):
    return ((row['ham_0'] + row['ham_1'] + row['ham_2'])/3 >= 0.5).astype(int)

df['cam'] = df.apply(cam,axis=1)
df['sam'] = df.apply(sam,axis=1)
df['ham'] = df.apply(ham,axis=1)
df.head()
# sam = df.apply(lambda x: np.logical_and(np.array(x['ham_0']), np.array(x['ham_1']), np.array(x['ham_2'])))

,text,label,text_tokens,ham_0,human_label_0,ham_1,human_label_1,ham_2,human_label_2,cam,sam,ham
0,"I don't think so. \n\nThis ""buffet"" is probabl...",0,"[I, don't, think, so., This, ""buffet"", is, pro...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, ...",no,"[0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, ...",no,"[0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, ...",no,"[0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, ...","[0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, ...","[0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, ..."
1,Ate here two times now and food has been excel...,1,"[Ate, here, two, times, now, and, food, has, b...","[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...",yes,"[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...",yes,"[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...",yes,"[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, ..."
2,Great food and great atmosphere. The one thing...,1,"[Great, food, and, great, atmosphere., The, on...","[1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",yes,"[1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, ...",yes,"[1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, ...",yes,"[1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, ..."
3,Obsessed. Atmosphere is a great combo of class...,1,"[Obsessed., Atmosphere, is, a, great, combo, o...","[1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, ...",yes,"[1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, ...",yes,"[1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, ...",yes,"[1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, ...","[1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, ...","[1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, ..."
4,I spend a lot of time trying as many burger jo...,1,"[I, spend, a, lot, of, time, trying, as, many,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",yes,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",yes,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",yes,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


<div class="alert alert-block alert-info">
    <b>Note:</b> By reconcatenating for spacy and re tokenize, do we obtain the coherent length?
</div>    

In [134]:
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
import spacy 

nlp = spacy.load('en_core_web_sm')
# soup = BeautifulSoup(html, 'html.parser')

def tokenize_ham(html):
    soup = BeautifulSoup(html, 'html.parser')
    tags = [str(tag.string) for tag in soup.find_all('span') if tag.string is not None]
    tokens = [str(tk.text) for doc in nlp.pipe(tags) for tk in doc]
    return tokens

files = [fpath for fpath in os.listdir(extracted_path) if fpath.endswith('.csv') and 'part7' not in fpath]

for fpath in tqdm(files, total=len(files)):
        
    df = pd.read_csv(path.join(extracted_path, fpath))

    dfs = [df.loc[0::3, df.columns != 'Answer.html_output'].reset_index(drop=True)]
    dfs += [df.loc[idx::3, ['Answer.html_output']].reset_index(drop=True).rename(columns={'Answer.html_output':'ham_html_'}).add_suffix(str(idx)) for idx in range(3) ]
    clean_df = pd.concat(dfs,axis=1).rename(columns={'Input.label': 'label', 'Input.text': 'text', 'Answer.Q1Answer':'human_label'})
    clean_df = clean_df[['text', 'ham_html_0', 'ham_html_1', 'ham_html_2', 'label', 'human_label']]

    clean_df['ham_tokens'] = clean_df['ham_html_0'].apply(tokenize_ham)
    clean_df['count_ham_tokens'] = clean_df['ham_tokens'].apply(lambda row: len(row))

    clean_df['text_tokens'] = [[tk.text for tk in doc if not tk.is_space] for doc in nlp.pipe(clean_df['text'].tolist())]
    clean_df['count_text_tokens'] = clean_df['text_tokens'].apply(lambda row: len(row))

    is_every_row_ok = (clean_df['count_text_tokens'] == clean_df['count_ham_tokens']).all()
    print(fpath, ': Same tokens between text and ham >', is_every_row_ok)

  0%|          | 0/6 [00:00<?, ?it/s]

ham_part5.csv : Same tokens between text and ham > True
ham_part8(200words).csv : Same tokens between text and ham > True
ham_part6(100words).csv : Same tokens between text and ham > True
ham_part4.csv : Same tokens between text and ham > True
ham_part3.csv : Same tokens between text and ham > True
ham_part1(50words).csv : Same tokens between text and ham > True


###### Split tokens in human attention maps by spacy, verify if `len(token) == len(annotation)`

In [ ]:
def binarize_ham(html):
    soup = BeautifulSoup(html, 'html.parser')
    tags = [tag for tag in soup.find_all('span') if tag.string is not None]
    
    tag_annot = [int('active' in t.get('class', [])) for t in tags]
    tag_str = [str(t.string) for t in tags]
    
    ham = []
    
    for annot, splitted_tokens in zip(tag_annot, nlp.pipe(tag_str)):
        annotation = [annot * int(not tk.is_punct) for tk in splitted_tokens]
        ham += annotation
    
    return ham

html = clean_df['ham_html_0']
print(html[0])
print(tokenize_ham(html[0]))

In [ ]:
files = [fpath for fpath in os.listdir(dataset_path) if fpath.endswith('.csv') and 'part7' not in fpath]

for fpath in tqdm(files, total=len(files)):
        
    df = pd.read_csv(path.join(dataset_path, fpath))

    dfs = [df.loc[0::3, df.columns != 'Answer.html_output'].reset_index(drop=True)]
    dfs += [df.loc[idx::3, ['Answer.html_output']].reset_index(drop=True).rename(columns={'Answer.html_output':'annotation_html_'}).add_suffix(str(idx)) for idx in range(3) ]
    clean_df = pd.concat(dfs,axis=1).rename(columns={'Input.label': 'label', 'Input.text': 'text', 'Answer.Q1Answer':'human_label'})
    clean_df = clean_df[['text', 'ham_html_0', 'ham_html_1', 'ham_html_2', 'label', 'human_label']]

    clean_df['ham_tokens'] = clean_df['ham_html_0'].apply(binarize_ham)
    clean_df['count_ham_tokens'] = clean_df['ham_tokens'].apply(lambda row: len(row))

    clean_df['text_tokens'] = [[tk.text for tk in doc if not tk.is_space] for doc in nlp.pipe(clean_df['text'].tolist())]
    clean_df['count_text_tokens'] = clean_df['text_tokens'].apply(lambda row: len(row))

    is_every_row_ok = (clean_df['count_text_tokens'] == clean_df['count_ham_tokens']).all()
    print(fpath, ': Same tokens between text and ham >', is_every_row_ok)

## Check sentence length

In [30]:
for fpath in tqdm(files, total=len(files)):
        
    df = pd.read_csv(path.join(dataset_path, fpath))
    display(HTML(f'<h3>{fpath}</h3>'))
    dfs = [df.loc[0::3, df.columns != 'Answer.html_output'].reset_index(drop=True)]
    dfs += [df.loc[idx::3, ['Answer.html_output']].reset_index(drop=True).rename(columns={'Answer.html_output':'ham_html_'}).add_suffix(str(idx)) for idx in range(3) ]
    clean_df = pd.concat(dfs,axis=1).rename(columns={'Input.label': 'label', 'Input.text': 'text', 'Answer.Q1Answer':'human_label'})
    clean_df = clean_df[['text', 'ham_html_0', 'ham_html_1', 'ham_html_2', 'label', 'human_label']]
    clean_df['text_tokens'] = [[tk.text for tk in doc if not tk.is_space] for doc in nlp.pipe(clean_df['text'].tolist())]
    
    clean_df['text_length'] = clean_df.text_tokens.str.len()
    display(clean_df.describe().transpose())

  0%|          | 0/6 [00:00<?, ?it/s]

,count,mean,std,min,25%,50%,75%,max
label,1000.0,0.516,0.499994,0.0,0.0,1.0,1.0,1.0
text_length,1000.0,71.694,7.983865,55.0,66.0,71.0,78.0,97.0


,count,mean,std,min,25%,50%,75%,max
label,181.0,0.508287,0.501318,0.0,0.0,1.0,1.0,1.0
text_length,181.0,230.071823,8.317340,210.0,224.0,230.0,235.0,252.0


,count,mean,std,min,25%,50%,75%,max
label,438.0,0.513699,0.500384,0.0,0.0,1.0,1.0,1.0
text_length,438.0,115.276256,5.565827,101.0,112.0,114.0,118.0,144.0


,count,mean,std,min,25%,50%,75%,max
label,1000.0,0.507,0.500201,0.0,0.0,1.0,1.0,1.0
text_length,1000.0,71.604,7.837641,55.0,65.0,71.0,78.0,101.0


,count,mean,std,min,25%,50%,75%,max
label,1000.0,0.497,0.500241,0.0,0.0,0.0,1.0,1.0
text_length,1000.0,71.958,8.178944,53.0,65.0,72.0,78.0,107.0


,count,mean,std,min,25%,50%,75%,max
label,300.0,0.483333,0.500557,0.0,0.0,0.0,1.0,1.0
text_length,300.0,58.006667,3.009454,50.0,56.0,57.0,60.0,72.0


### Handle case part7.csv

In [156]:
df = pd.read_csv(path.join(extracted_path, 'ham_part7.csv'))
df['ham_html_'] = df[f'Answer.html_output'].apply(lambda x: yelp_hat_ham(x, nlp))
display(HTML('<h3>Part 7</h3>'))
display(df.head())

duplicates = df.groupby(df['Input.text'].tolist(),as_index=False).size()
display(HTML('<h3>Duplication du part 7</h3>'))
display(duplicates.head())

,Input.label,Input.text,Answer.Q1Answer,Answer.html_output,ham_html_
0,0,*knocks on door*\n*walks into restaurant*\n*go...,no,<span>*knocks</span> <span>on</span> <span>doo...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,0,*knocks on door*\n*walks into restaurant*\n*go...,no,<span>*knocks</span> <span>on</span> <span>doo...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,0,*knocks on door*\n*walks into restaurant*\n*go...,no,<span>*knocks</span> <span>on</span> <span>doo...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,0,$18.95 per person....Was really hungry for lun...,no,<span>$18.95</span> <span>per</span> <span>per...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,0,$18.95 per person....Was really hungry for lun...,no,<span>$18.95</span> <span>per</span> <span>per...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


,index,size
0,$18.95 per person....Was really hungry for lun...,3
1,*knocks on door*\n*walks into restaurant*\n*go...,3
2,1. Happy hour pricing is only available in the...,3
3,20 Minute wait per dish. Never coming here aga...,3
4,"2nd time eating here, We drive 35 miles from H...",3


In [149]:
dupli_2 = duplicates[duplicates['size'] < 3]

for d in tqdm(dupli_2['index'][:3]):
    display(df.loc[df['Input.text'] == d])
    
for dup_rows in [2, 3, 4]:
    print('Duplicate of', dup_rows, ':', sum(duplicates['size'] == dup_rows))

  0%|          | 0/1 [00:00<?, ?it/s]

KeyError: 'Input.text'

In [150]:
# Drop all text that has only 2 reviews:
df2 = df[~df['Input.text'].isin(dupli_2['index'])]

# Verify if no more 2-row duplicate exist
duplicates_2 = df2.groupby(df2['Input.text'].tolist(),as_index=False).size()
display(duplicates_2['size'].unique())
print('>> Expected not to have "2" after drop')

KeyError: 'Input.text'

In [151]:
duplicates_2['size'].unique() == [3]

NameError: name 'duplicates_2' is not defined

In [72]:
# Drop 4th annotation
df3 = df2.groupby('Input.text').head(3).reset_index(drop=True)

# Check if no more duplicate of 4:
duplicates_3 = df3.groupby(df3['Input.text'].tolist(),as_index=False).size()
display(duplicates_3['size'].unique())

print('Check correctly remove number of duplicate of 4', len(df3) + len(text_4) == len(df2))

NameError: name 'df2' is not defined

In [60]:
clean_df = df3

In [69]:
duplicates_3['size'].unique()[0] == 3

True

## Test dataset

In [51]:
from data import YelpHat

yelp_hat = YelpHat(root=path.join(cache_path, 'dataset'))

Reformat /Users/dunguyen/Projects/explanation_on_pair_sequences_task/notebooks/../.cache/dataset/yelp-hat/caching/ham_part7.csv


Drop 12 samples because HAMs are not compatibles


Reformat /Users/dunguyen/Projects/explanation_on_pair_sequences_task/notebooks/../.cache/dataset/yelp-hat/caching/ham_part5.csv
Reformat /Users/dunguyen/Projects/explanation_on_pair_sequences_task/notebooks/../.cache/dataset/yelp-hat/caching/ham_part8(200words).csv


Drop 2 samples because HAMs are not compatibles


Reformat /Users/dunguyen/Projects/explanation_on_pair_sequences_task/notebooks/../.cache/dataset/yelp-hat/caching/ham_part6(100words).csv


Drop 1 samples because HAMs are not compatibles


Reformat /Users/dunguyen/Projects/explanation_on_pair_sequences_task/notebooks/../.cache/dataset/yelp-hat/caching/ham_part4.csv
Reformat /Users/dunguyen/Projects/explanation_on_pair_sequences_task/notebooks/../.cache/dataset/yelp-hat/caching/ham_part3.csv
Reformat /Users/dunguyen/Projects/explanation_on_pair_sequences_task/notebooks/../.cache/dataset/yelp-hat/caching/ham_part1(50words).csv


In [53]:
len(yelp_hat)

3482

In [55]:
from torch.utils.data import random_split

N_train = int(len(yelp_hat) * 0.7)
N_val = len(yelp_hat) - N_train
trainset, valset = random_split(yelp_hat, [N_train, N_val])

In [57]:
len(trainset)

2437